In [1]:
import os
import cv2
import numpy as np
import pandas as pd

In [2]:
class BoundBox:
    def __init__(self, ymin, xmin , ymax , xmax, objness=None, classes=None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        self.objness = objness
        self.classes = classes
        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)

        return self.label

    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]

        return self.score
    
    
# x1,y1 ----------
#       |        |
#       |        |
#       |        |
#       ----------x2,y2
def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b
    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2, x4) - x1
    else:
        if x2 < x3:
            return 0
        else:
            return min(x2, x4) - x3

# IoU = (A intersection B) / (A union B)
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap(
        [box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap(
        [box1.ymin, box1.ymax], [box2.ymin, box2.ymax])
    intersect = intersect_w * intersect_h
    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    union = w1*h1 + w2*h2 - intersect
    if union == 0: 
        return 0
    return float(intersect) / union


def create_box(a):
    return BoundBox(a[0],a[1],a[2],a[3]) # y1, x1, y2, x2 


![Getting Started](./mark_down/image/confusion-matrix.webp)

In [5]:
# row different together
def matrix_confusion(matrix):
    convert =[]
    amount_row = len(matrix)
    amount_col = len(matrix[0])
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            convert.append([i,j,matrix[i][j]])
    df = pd.DataFrame(convert, columns=['row','col','score'])
    # print(df)
    # convert row, col, score to matrix
    convert = df.values
    convert = convert.tolist()
    convert = sorted(convert, key=lambda x: x[2], reverse=True)
    # print(convert)
    
    dict_col = []
    dict_row = []
    values = []
    for i in range(len(convert)):
        if str(convert[i][0]) not in dict_row and str(convert[i][1]) not in dict_col:
            dict_col.append(str(convert[i][1]))
            dict_row.append(str(convert[i][0]))
            values.append(float(convert[i][2]))
    
    values = pd.DataFrame({'row':dict_row, 'col':dict_col, 'score':values})
    # if IoU ≥0.5, classify the object detection as True Positive(TP)
    acc_values = values.loc[values['score'] >= 0.5]
    TP = len(acc_values)
    # if Iou <0.5, then it is a wrong detection and classify it as False Positive(FP)
    FP = len(values) - len(acc_values)
    if amount_col > amount_row:
        FP = FP + (amount_col - amount_row)
    # When a ground truth is present in the image and model failed to detect the object, classify it as False Negative(FN).
    FN = amount_row - amount_col
    # True Negative (TN): TN is every part of the image where we did not predict an object. This metrics is not useful for object detection, hence we ignore TN.
    return TP, FP, FN

def precision_recall(TP, FP, FN):
    # precision = TP / (TP + FP)
    # recall = TP / (TP + FN)
    if TP + FN == 0 and TP + FP ==0:
        return 0,0
    if TP + FP == 0:
        return 0,TP / (TP + FN)
    
    if TP + FN == 0:
        return TP / (TP + FP), 0
    return TP / (TP + FP), TP / (TP + FN)

def f1_score(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def accuracy(TP, FP, FN):
    if TP + FP + FN == 0:
        return 0
    return TP / (TP + FP + FN)

def scrore_frame_x(dict_trust,dict_predict):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    acc_ls = []
    f1_score_ls = []
    precision_ls = []
    recall_ls = []
    for key in dict_trust:
        if key in dict_predict:
            matrix_score = []
            
            for i in range(len(dict_trust[key])):
                ls = []
                for j in range(len(dict_predict[key])):
                    ls.append(bbox_iou(create_box(dict_trust[key][i]),create_box(dict_predict[key][j])))
                    
                matrix_score.append(ls)
                
            # matrix_score = np.array(matrix_score)
            # matrix_score = pd.DataFrame(matrix_score)
            # matrix_score.to_csv('./matrix_score2.csv')
            TP, FP, FN = matrix_confusion(matrix_score)
            acc_ls.append(accuracy(TP, FP, FN))
            precision_ls.append(precision_recall(TP, FP, FN)[0])
            recall_ls.append(precision_recall(TP, FP, FN)[1])
            f1_score_ls.append(f1_score(precision_recall(TP, FP, FN)[0],precision_recall(TP, FP, FN)[1]))
            print(TP,FP,FN)
    return np.mean(acc_ls), np.mean(precision_ls), np.mean(recall_ls), np.mean(f1_score_ls)       
    

def evaluate_count_person(file_trust,file_predict):
    df_trust = pd.read_csv(file_trust)
    df_predict = pd.read_csv(file_predict)
    scores = []
    for frame in range(len(df_trust['frame_x'])):
    # print(df_trust['bounding_box'][0])
    # print(df_predict['bounding_box'][0])        
        import ast
        dict_trust = ast.literal_eval(df_trust['bounding_box'][frame])
        dict_predict = ast.literal_eval(df_predict['bounding_box'][frame])
        scores.append(scrore_frame_x(dict_trust,dict_predict))
        
    return np.mean(scores[0]), np.mean(scores[1]), np.mean(scores[2]), np.mean(scores[3])

evaluate_count_person('./trust_csv/1.csv','./predict_csv/1.csv')

14 7 88
16 7 74
19 8 61
20 3 62
12 9 65
22 8 60
12 4 69
11 11 68
18 4 62
16 5 60
17 7 59
11 4 64
4 7 77
10 10 61
7 10 67
10 13 61
7 17 60
10 11 63
12 10 62
10 9 60
10 15 56
5 13 66
12 15 55
11 13 60
11 7 65
10 11 58
12 11 60
13 12 61
11 12 63
10 19 51
17 10 52
6 14 55
5 13 56
7 11 56
6 10 59
7 13 54
4 15 60
10 13 62
5 7 69
10 8 62
3 7 70
3 11 70
7 14 64
11 7 62
14 6 57
15 7 56
11 8 57
9 8 63
12 4 60
5 4 42
9 10 21
7 16 15
5 12 22
8 7 34
9 8 25
4 12 26
2 17 23
11 11 20
10 6 34
3 5 9
2 10 5
0 9 11
3 12 8
3 12 11
4 11 5
4 8 9
4 12 5
3 13 3
3 8 6
2 9 4
4 9 3
2 7 9
4 8 2
6 8 5
3 9 1
1 11 4
3 10 3
2 8 4
4 10 2
5 4 6
4 5 7
3 8 5
3 9 4
2 7 8
5 4 5
8 5 3
12 1 10
4 10 2
6 8 3
4 11 -3
4 8 1
6 7 1
5 8 3
4 3 2
3 4 3
2 6 2
3 7 1
3 6 4
5 6 1
6 6 1
1 9 -1
1 9 -2
2 6 0
2 4 1
1 7 3
3 9 -1
3 10 -2
4 6 6
5 7 5
4 8 6
2 11 4
5 5 2
8 1 13
6 2 13
5 2 13
5 4 7
7 3 7
7 5 10
8 4 6
9 5 11
11 6 6
8 3 8
10 5 4
10 6 7
9 6 6
9 6 4
10 5 6
9 6 4
9 1 39
7 2 16
8 0 27
8 1 27
11 1 20
14 0 15
10 2 22
10 1 25
7 3 19
10 1 19

(0.290001053005673,
 0.3303910615017583,
 0.3780632453821706,
 0.43242853875378356,
 0.27792639254433277)

In [9]:
# row different together
def matrix_(matrix):
    convert =[]
    amount_row = len(matrix)
    amount_col = len(matrix[0])
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            convert.append([i,j,matrix[i][j]])
    df = pd.DataFrame(convert, columns=['row','col','score'])
    # print(df)
    # convert row, col, score to matrix
    convert = df.values
    convert = convert.tolist()
    convert = sorted(convert, key=lambda x: x[2], reverse=True)
    # print(convert)
    
    dict_col = []
    dict_row = []
    values = []
    for i in range(len(convert)):
        if str(convert[i][0]) not in dict_row and str(convert[i][1]) not in dict_col:
            dict_col.append(str(convert[i][1]))
            dict_row.append(str(convert[i][0]))
            values.append(float(convert[i][2]))
    
    values = pd.DataFrame({'row':dict_row, 'col':dict_col, 'score':values})
    # if IoU ≥0.5, classify the object detection as True Positive(TP)
    acc_values = values.loc[values['score'] >= 0.5]
    TP = len(acc_values)
    # if Iou <0.5, then it is a wrong detection and classify it as False Positive(FP)
    FP = len(values) - len(acc_values)
    if amount_col > amount_row:
        FP = FP + (amount_col - amount_row)
    # When a ground truth is present in the image and model failed to detect the object, classify it as False Negative(FN).
    FN = amount_row - amount_col
    # True Negative (TN): TN is every part of the image where we did not predict an object. This metrics is not useful for object detection, hence we ignore TN.
    return TP, FP, FN

def confusion_matrix_frame_x(dict_trust,dict_predict):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    matrix_score = []
    for key_t in dict_trust:
        # if key in dict_predict:
        
        
        for i in range(len(dict_trust[key_t])):
            ls = []
            for key_p in dict_predict:
                for j in range(len(dict_predict[key_p])):
                    ls.append([key_t,key_p,bbox_iou(create_box(dict_trust[key_t][i]),create_box(dict_predict[key_p][j]))])

            matrix_score.append(ls)
        # matrix_score = np.array(matrix_score)
        # print(matrix_score)
        # matrix_score = pd.DataFrame(matrix_score)
        # matrix_score.to_csv('./matrix_score1.csv')
        TP, FP, FN = matrix_(matrix_score)
        print(TP, FP, FN)
            
    

def evaluate_count_person(file_trust,file_predict):
    df_trust = pd.read_csv(file_trust)
    df_predict = pd.read_csv(file_predict)
    # for frame in range(len(df_trust['frame_x'])):
    # print(df_trust['bounding_box'][0])
    # print(df_predict['bounding_box'][0])        
    import ast
    dict_trust = ast.literal_eval(df_trust['bounding_box'][0])
    dict_predict = ast.literal_eval(df_predict['bounding_box'][0])
    confusion_matrix_frame_x(dict_trust,dict_predict)
    
evaluate_count_person('./1.csv','./1.csv')